In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
random_state = 7
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def model_validation(tr,ts, iterations, learning_rate,depth, n_splits = 10):
   
    # get current directory
    path = os.getcwd()
    parent = os.path.dirname(path)

    train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
    test = pd.read_csv(parent + '\Data\\' + ts + ".csv")
    
    X_train = train.drop("target", axis=1)
    y_train = train["target"]
    
    X_test  = test.drop("target", axis=1)
    y_test = test["target"]
    
        # initialize Pool
    train_pool = Pool(X_train, 
                      y_train)
        # initialize Pool
    test_pool = Pool(X_test, 
                      y_test)
    
    scores = pd.DataFrame(columns=["Iterations", "Learning Rate", "Depth", "F1", "ROC", "Sensitivity", "Specificty", "CV F1", "CV ROC", "ROC Sens", "ROC Spec"])
    
#     from sklearn.linear_model import LogisticRegression
#     clf = LogisticRegression(random_state=0, penalty="l1", solver='liblinear')
#     clf.fit(X_train, y_train)
#     pred = clf.predict(X_test)
#     f1 = (f1_score(pred, y_test))

#     cm = confusion_matrix(y_test, pred)
#     total=sum(sum(cm))
#     roca = round(roc_auc_score(y_test, pred),4)
#     sens = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
#     spec = round(cm[0,0]/(cm[1,0]+cm[0,0]),4)      
    
#     print(f1, roca, sens, spec)

    for iteration in iterations:
        for rate in learning_rate:
            for deep in depth:
                    start = time.time()
                    model = CatBoostClassifier(iterations=iteration,
                           depth=deep,
                           learning_rate=rate,
                           loss_function='Logloss',
                           verbose=False)
                    
                    model.fit(train_pool)
                    pred = model.predict(X_test)
                    f1 = (f1_score(pred, y_test))
                    
                    cm = confusion_matrix(y_test, pred)
                    total=sum(sum(cm))
                    roca = round(roc_auc_score(y_test, pred),4)
                    sens = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
                    spec = round(cm[0,0]/(cm[1,0]+cm[0,0]),4)                    
                    
                    
                    
                    scores_cv = []
                    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
                    skf.get_n_splits(X_train, y_train)
                    for train_index, test_index in skf.split(X_train,y_train):
                        X_cv_train, X_cv_test = X_train.iloc[train_index], X_train.iloc[test_index]
                        y_cv_train, y_cv_test = y_train.iloc[train_index], y_train.iloc[test_index]

                        model_cv = CatBoostClassifier(iterations=iteration,
                                               depth=deep,
                                               learning_rate=rate,
                                               loss_function='Logloss',
                                               verbose=False)

                        # initialize Pool
                        train_pool = Pool(X_cv_train, 
                                          y_cv_train)
                            # initialize Pool
                        test_pool = Pool(X_cv_test, 
                                          y_cv_test)

                        model_cv.fit(train_pool)
                        pred = model_cv.predict(X_cv_test)
                        f1_cv = (f1_score(pred, y_cv_test))

                        cm = confusion_matrix(y_cv_test, pred)
                        total=sum(sum(cm))
                        roca_cv = round(roc_auc_score(y_cv_test, pred),4)
                        sens_cv = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
                        spec_cv = round(cm[0,0]/(cm[1,0]+cm[0,0]),4) 
                        
                        scores_cv.append([f1_cv, roca_cv, sens_cv, spec_cv])

                      
                    
                    mean = (np.mean(np.array(scores_cv), axis=0))
                    df_new_row = pd.DataFrame(data=([[iteration, rate, deep, f1, roca, sens, spec, mean[0], mean[1], mean[2], mean[3]]]), 
                                              columns=["Iterations", "Learning Rate", "Depth",  "F1", "ROC", "Sensitivity", "Specificty",
                                                      "CV F1", "CV ROC", "ROC Sens", "ROC Spec"])
                    scores = pd.concat([scores,df_new_row], ignore_index=True)
                    end = time.time()
                    print(end - start)
                    
            
    return scores

In [7]:
tr = "testset_ALL_vs_Healthy_reduced_25_20_target_in"
ts = "trainset_ALL_vs_Healthy_reduced_25_80_target_in"

# tr = "trainset_AML_vs_ALL_reduced_90_target_in"
# ts = "testset_AML_vs_ALL_reduced_90_target_in"

start = time.time()

#  iterations, learning_rate,depth, weights, n_splits = 10
results = model_validation(tr,ts, [100 ,200 ,300 ,400], [0.3, 0.5, 0.7, 1],[3, 4, 5, 6, 7])




2.466296911239624
2.316720962524414
1.9144949913024902
3.454414129257202
4.998201847076416
1.940497636795044
1.999420166015625
2.124171018600464
4.089534759521484
5.019494533538818
1.6701984405517578
2.519388437271118
2.505042314529419
3.5024874210357666
5.47066068649292
1.958678960800171
3.352480411529541
2.492325782775879
4.387136697769165
5.815312623977661
4.176868915557861
4.0832953453063965
6.635437726974487
6.183451175689697
11.246460437774658
2.239237070083618
4.399917125701904
5.192286729812622
6.730186700820923
10.79770016670227
4.172631502151489
4.196710586547852
6.5931456089019775
7.555797338485718
11.057019472122192
3.3422868251800537
3.916666030883789
5.79100489616394
6.345988988876343
8.960995197296143
5.138134002685547
4.750445365905762
6.59511661529541
7.941047191619873
14.079285621643066
3.8474740982055664
5.804046630859375
6.644202470779419
8.433842658996582
14.405434846878052
4.666889429092407
4.890111207962036
7.206995487213135
8.235708475112915
15.35604190826416
4.

In [8]:
index = 0
top = 0
i = 0
for row in results.loc[:]["CV F1"]:
    if row > top:
        top = row
        index = i
    i = i + 1
print(results.loc[index])
for item in results.loc[index]:
    print(item)

Iterations            100
Learning Rate         0.3
Depth                   3
F1               0.991703
ROC                  0.98
Sensitivity        0.9917
Specificty         0.9683
CV F1            0.993534
CV ROC             0.9853
ROC Sens          0.99434
ROC Spec           0.9736
Name: 0, dtype: object
100
0.3
3
0.9917027417027418
0.98
0.9917
0.9683
0.9935342589507868
0.9853
0.99434
0.9736
